In [ ]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 랜덤 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
from PIL import Image  # 이미지 처리 및 파일 입출력을 위한 라이브러리
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 생성기 유틸리티
from keras.models import Sequential, Model  # Keras의 시퀀셜 및 함수형 API 모델
from keras.layers import (Dense, Dropout, Flatten, Conv2D, MaxPooling2D, 
                          BatchNormalization, Concatenate, AveragePooling2D, Input)  # 신경망의 레이어 구성 요소
from keras.optimizers import SGD  # 최적화 알고리즘
from keras.callbacks import EarlyStopping, ModelCheckpoint  # 학습 중 콜백 함수

def generators(train_dir, val_dir):
    """학습 및 검증 데이터를 생성하는 함수"""
    # 학습 데이터 생성기 설정
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 최대 180도 회전
                                       width_shift_range=0.2,  # 가로 방향 이동
                                       height_shift_range=0.2,  # 세로 방향 이동
                                       shear_range=0.2,  # 시어 변환
                                       zoom_range=0.2,  # 줌 인/줌 아웃
                                       horizontal_flip=True,  # 가로 방향 뒤집기
                                       vertical_flip=True,  # 세로 방향 뒤집기
                                       fill_mode='nearest')  # 변환 중 생기는 빈 공간을 주변의 유사한 픽셀 값으로 채우기

    # 검증 데이터 생성기 설정
    val_datagen = ImageDataGenerator(rescale=1/255)  # 이미지의 픽셀 값을 0-1 범위로 정규화

    # 학습 데이터 생성
    train_generator = train_datagen.flow_from_directory(train_dir,  # 학습 데이터가 위치한 디렉토리
                                                        target_size=(227, 227),  # 입력 이미지 크기
                                                        batch_size=512,  # 배치 크기
                                                        class_mode='categorical',  # 다중 클래스 분류
                                                        shuffle=True)  # 데이터를 섞어서 배치 생성

    # 검증 데이터 생성
    val_generator = val_datagen.flow_from_directory(val_dir,  # 검증 데이터가 위치한 디렉토리
                                                    target_size=(227, 227),  # 입력 이미지 크기
                                                    batch_size=512,  # 배치 크기
                                                    class_mode='categorical')  # 다중 클래스 분류

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

# 학습 및 검증 데이터 디렉토리 설정
train_dir = 'your_path'  # 학습 데이터 디렉토리 경로
val_dir = 'your_path'  # 검증 데이터 디렉토리 경로

# 데이터 생성기 생성
train_generator, val_generator = generators(train_dir, val_dir)

def inception(x, filters):
    """GoogLeNet의 Inception 모듈을 정의하는 함수"""
    # 1x1 Conv
    path1 = Conv2D(filters=filters[0], kernel_size=(1, 1), strides=1, padding='same', activation='relu')(x)
    
    # 1x1 Conv -> 3x3 Conv
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1, 1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3, 3), strides=1, padding='same', activation='relu')(path2)
    
    # 1x1 Conv -> 5x5 Conv
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1, 1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5, 5), strides=1, padding='same', activation='relu')(path3)
    
    # 3x3 MaxPooling -> 1x1 Conv
    path4 = MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1, 1), strides=1, padding='same', activation='relu')(path4)
    
    # 각 경로의 결과를 Concatenate로 병합
    return Concatenate(axis=-1)([path1, path2, path3, path4])

def auxiliary(x, name=None):
    """GoogLeNet의 Auxiliary Classifier를 정의하는 함수"""
    layer = AveragePooling2D((5, 5), strides=3, padding='valid')(x)  # 5x5 AveragePooling
    layer = Conv2D(128, (1, 1), 1, padding='same', activation='relu')(layer)  # 1x1 Conv
    layer = Flatten()(layer)  # 평탄화
    layer = Dense(256, activation='relu')(layer)  # FC Layer 1
    layer = Dropout(0.4)(layer)  # 드롭아웃
    layer = Dense(1000, activation='softmax', name=name)(layer)  # FC Layer 2 및 softmax 활성화 함수
    return layer

def GoogLeNet():
    """GoogLeNet 모델 정의"""
    layer_in = Input(shape=(224, 224, 3))  # 입력 레이어
    layer = Conv2D(filters=64, kernel_size=(7, 7), strides=2, padding='same', activation='relu')(layer_in)  # 7x7 Conv
    layer = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(layer)  # 3x3 MaxPooling
    layer = BatchNormalization()(layer)  # 배치 정규화

    layer = Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding='same', activation='relu')(layer)  # 1x1 Conv
    layer = Conv2D(filters=192, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(layer)  # 3x3 Conv
    layer = BatchNormalization()(layer)  # 배치 정규화
    layer = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(layer)  # 3x3 MaxPooling

    layer = inception(layer, [64, (96, 128), (16, 32), 32])  # Inception 모듈
    layer = inception(layer, [128, (128, 192), (32, 96), 64])  # Inception 모듈
    layer = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(layer)  # 3x3 MaxPooling

    layer = inception(layer, [192, (96, 208), (16, 48), 64])  # Inception 모듈
    aux1 = auxiliary(layer, name='aux1')  # 첫 번째 Auxiliary Classifier
    layer = inception(layer, [160, (112, 224), (24, 64), 64])  # Inception 모듈
    layer = inception(layer, [128, (128, 256), (24, 64), 64])  # Inception 모듈
    layer = inception(layer, [112, (144, 288), (32, 64), 64])  # Inception 모듈
    aux2 = auxiliary(layer, name='aux2')  # 두 번째 Auxiliary Classifier
    layer = inception(layer, [256, (160, 320), (32, 128), 128])  # Inception 모듈
    layer = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(layer)  # 3x3 MaxPooling

    layer = inception(layer, [256, (160, 320), (32, 128), 128])  # Inception 모듈
    layer = inception(layer, [384, (192, 384), (48, 128), 128])  # Inception 모듈
    layer = AveragePooling2D(pool_size=(7, 7), strides=1, padding='valid')(layer)  # 7x7 AveragePooling

    layer = Flatten()(layer)  # 평탄화
    layer = Dropout(0.4)(layer)  # 드롭아웃
    layer = Dense(units=256, activation='linear')(layer)  # FC Layer 1
    main = Dense(units=1000, activation='softmax', name='main')(layer)  # FC Layer 2 및 softmax 활성화 함수

    model = Model(inputs=layer_in, outputs=[main, aux1, aux2])  # GoogLeNet 모델 정의
    model.compile(optimizer=SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])  # 모델 컴파일
    model.summary()  # 모델 구조 출력

    return model

# GoogLeNet 모델 생성
model = GoogLeNet()

# 모델 저장 경로 및 콜백 설정
model_path = 'your_path'  # 최적의 모델을 저장할 파일 경로
CP = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)  # 최적의 모델을 저장하는 콜백
ES = EarlyStopping(monitor='val_loss', patience=10)  # 조기 종료를 위한 콜백, 10 에포크 동안 개선이 없으면 학습 중단

# 모델 학습
history = model.fit(train_generator,  # 학습 데이터 생성기
                    epochs=100,  # 최대 100 에포크 동안 학습
                    validation_data=val_generator,  # 검증 데이터 생성기
                    callbacks=[CP, ES])  # 콜백 함수들

# 학습 결과 시각화
plt.plot(history.history['loss'])  # 학습 손실 그래프
plt.plot(history.history['accuracy'])  # 학습 정확도 그래프
plt.plot(history.history['val_loss'])  # 검증 손실 그래프
plt.plot(history.history['val_accuracy'])  # 검증 정확도 그래프
plt.title('GoogLeNet')  # 그래프 제목
plt.ylabel('loss')  # y축 라벨
plt.xlabel('epoch')  # x축 라벨
plt.legend(['loss', 'val_loss', 'accuracy', 'val_accuracy'], loc='upper left')  # 범례
plt.show()  # 그래프 출력

# 최적의 검증 정확도와 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))  # 최고 검증 정확도 출력
print('Best Val Loss:', min(history.history['val_loss']))  # 최저 검증 손실 출력